#### Premessa per tutte le strategie: i calcoli vengono fatti di sera con prezzo di chiusura

Modifica la classe IterativeBackTest per tenere conto del fatto che noi non possiamo fare le operazioni al tempo t,
dato che il mercato è chiuso perché abbiamo appena utilizzato i prezzi di chiusura. Pertanto usiamo i prezzi di chiusura 
per fare tutti i calcoli, ma facciamo le operazioni utilizzando i prezzi di apertura del giorno dopo. Questo lo facciamo 
per tutto tranne che per la funzione close, che resta con la chiusura a fine giornata per simulare il fatto che abbiamo 
deciso di chiudere tutto e andare in vacanza e quindi lo facciamo alla sera.

<font color=RED>Dato che a lezione abbiamo già fatto la prima parte, quella che utilizza i prezzo di apertura del tempo
t+1, voi dovete solo fare la seconda parte cioè fare in modo che la funziona close chiuda le posizioni ai prezzi di 
chiusura del tempo t. Questa funzione close viene chiamata soltanto alla fine. In questo modo possiamo far andare il loop
fino al tempo len(self.data)-1 invece che -2</font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")
from itertools import product

In [2]:
import tpqoa
api = tpqoa.tpqoa("oanda.cfg")

In [3]:
# la logica per risolvere questo problema: introdurre un boolean nella classe che decide se 
# le transazione dovrebbero veinre fatte in regime di "closure", cioe con i prezzi bid/ask di chiusura dello stesso
# giorno o no. il boolean communichera al metodo buy sell quale prezzo debbono usare e da quale giorno prenderli, e
# per default eh negativo. ci si puo fare close solo una volta per classe (le vacanze sono solo una volta allanno)
# Non ho fatto questa classe come figlia di interative base fatta in classe, perche li cera pocco da riutilizzare

class IterativeBase():

    def __init__(self, symbol, start, end, bilancio, fee=(0,0,0)):
        #super().__init__()
        #fee(x,y,z) x percentuale, comm minima, comm massima.
        self.symbol = symbol
        self.start = start
        self.end = end
        self.initial_balance = bilancio
        self.current_balance = bilancio
        self.units= 0 #quanti euro HO
        self.trades=0
        self.fee=fee
        self.get_data()
        self.closure=False
        
    def get_data(self): # carica i dati
        df=pd.DataFrame()
        df["bid"]=api.get_history(instrument=self.symbol, start=self.start, end=self.end , granularity = "D", price = "B")["c"]
        df["ask"]=api.get_history(instrument=self.symbol, start=self.start, end=self.end , granularity = "D", price = "A")["c"]
        df["Abid"]=api.get_history(instrument=self.symbol, start=self.start, end=self.end , granularity = "D", price = "B")["o"]
        df["Aask"]=api.get_history(instrument=self.symbol, start=self.start, end=self.end , granularity = "D", price = "A")["o"]
        df["prezzo"] = (df.ask+df.bid)/2
        df["logRet"]=np.log(df.prezzo/df.prezzo.shift(1)) 
        self.data=df
        
# compro a ask, vendo a bid, altrimenti debbo simulare lo spread
    def plot_data(self, cols=None):
        if cols is None:
            cols="prezzo"
        self.data[cols].plot(figsize=(25,15), title=self.symbol)
        
    def get_values(self,t):
        date=str(self.data.index[t])
        bid=self.data.bid[t]
        ask=self.data.ask[t]
        Aask=self.data.Aask[t]
        Abid=self.data.Abid[t]
        prezzo=self.data.prezzo[t]
        return date, prezzo, bid, ask, Abid, Aask
    
    def buy(self,t,units=None,amount=None):
        if self.closure is False:
            Adate, prezzo, bid, ask, Abid, Aask=self.get_values(t+1)
            if amount is not None:
                units= int(amount/Aask)   #sto comprando
            self.current_balance-= units*Aask       #riduco i miei soldi, sarebbe unidea controllare che io cho i soldi
            commissioni= min(max(units*Aask*self.fee[0],self.fee[1]),self.fee[2])
            self.current_balance-=commissioni
            self.units+=units
            self.trades+=1
            print(str(Adate),"Compro "+str(units) + ", al prezzo: " + str(Aask) + " spendendo " +str((units*Aask)))
        else:
            date, prezzo, bid, ask, Abid, Aask=self.get_values(t)
            if amount is not None:
                units= int(amount/ask)   #sto comprando
            self.current_balance-= units*ask       #riduco i miei soldi, sarebbe unidea controllare che io cho i soldi
            commissioni= min(max(units*ask*self.fee[0],self.fee[1]),self.fee[2])
            self.current_balance-=commissioni
            self.units+=units
            self.trades+=1
            print(str(date),"Compro "+str(units) + ", al prezzo: " + str(ask) + " spendendo " +str((units*ask)))
        
    def sell(self,t,units=None,amount=None): # units eh positivo 
        if self.closure is False:
            Adate, prezzo, bid, ask, Abid, Aask=self.get_values(t+1)
            if amount is not None:
                units= int(amount/Abid)   #sto comprando
            self.current_balance+= units*Abid       
            commissioni= min(max(units*Abid*self.fee[0],self.fee[1]),self.fee[2])
            self.current_balance-=commissioni
            self.units-=units
            self.trades+=1
            print(str(Adate),"Vendo "+str(units) + ", al prezzo: " + str(Abid) + " ricevendo " +str((units*Abid)))
        else:
            date, prezzo, bid, ask, Abid, Aask=self.get_values(t)
            if amount is not None:
                units= int(amount/bid)   #sto comprando
            self.current_balance+= units*bid      
            commissioni= min(max(units*bid*self.fee[0],self.fee[1]),self.fee[2])
            self.current_balance-=commissioni
            self.units-=units
            self.trades+=1
            print(str(date),"Vendo "+str(units) + ", al prezzo: " + str(bid) + " ricevendo " +str((units*bid)))
        
    def close(self,t):
        self.closure=True
        Adate, prezzo, bid, ask, Abid, Aask=self.get_values(t)
        if self.units>0:           #ho posizione lunga
            self.sell(t,units=self.units)
        elif self.units==0:
            print("Non ce posizione da chiudere!")
        else:
            self.buy(t,units=-self.units) # attenzione al meno
        performance=(self.current_balance -self.initial_balance)/self.initial_balance
        print("------CHIUSO TUTO--------")
        print(str(Adate)+ " BILANCIO: " + str(self.current_balance) + " PERFORMANCE: "
             + str(round(performance*100,2))+"%" + ", TRADE TOTALI: " + str(self.trades))
        if self.units>0:           #ho posizione lunga
            self.sell(t,units=self.units)
        else:
            self.buy(t,units=-self.units) # attenzione al meno
        performance=(self.current_balance -self.initial_balance)/self.initial_balance
        print("------CHIUSO TUTO--------")
        print(str(Adate)+ " BILANCIO: " + str(self.current_balance) + " PERFORMANCE: "
             + str(performance) + ", TRADE TOTALI: " + str(self.trades))
    
    def print_current_situation(self,t):
        date, prezzo, bid, ask, Abid, Aask=self.get_values(t)
        if self.units>0:
            valore=self.units*Abid
        else:
            valore=self.units*Aask
        print(str(date)+ " INVESTITI: " +str(self.units)+ " per un totale di "+ str(valore))


In [4]:
bc=IterativeBase("EUR_USD","2009-01-01","2021-10-30",100000)

In [5]:
bc.print_current_situation(50)

2009-02-24 22:00:00 INVESTITI: 0 per un totale di 0.0


In [6]:
bc.get_values(50)

('2009-02-24 22:00:00', 1.272335, 1.27221, 1.27246, 1.28453, 1.28478)

In [7]:
bc.data.iloc[50]

bid       1.272210
ask       1.272460
Abid      1.284530
Aask      1.284780
prezzo    1.272335
logRet   -0.009636
Name: 2009-02-24 22:00:00, dtype: float64

In [8]:
bc.data.iloc[51]

bid       1.274330
ask       1.274580
Abid      1.272210
Aask      1.272460
prezzo    1.274455
logRet    0.001665
Name: 2009-02-25 22:00:00, dtype: float64

In [9]:
bc.buy(50,units=1000)

2009-02-25 22:00:00 Compro 1000, al prezzo: 1.27246 spendendo 1272.46


In [10]:
bc.data.iloc[55]

bid       1.257650
ask       1.257900
Abid      1.262800
Aask      1.263800
prezzo    1.257775
logRet   -0.004264
Name: 2009-03-01 22:00:00, dtype: float64

In [11]:
bc.data.iloc[56]

bid       1.256090
ask       1.256340
Abid      1.256890
Aask      1.257140
prezzo    1.256215
logRet   -0.001241
Name: 2009-03-02 22:00:00, dtype: float64

In [12]:
bc.close(55)

2009-03-01 22:00:00 Vendo 1000, al prezzo: 1.25765 ricevendo 1257.6499999999999
------CHIUSO TUTO--------
2009-03-01 22:00:00 BILANCIO: 99985.18999999999 PERFORMANCE: -0.01%, TRADE TOTALI: 2


In [13]:
bc.data.bid

time
2008-12-31 22:00:00    1.40390
2009-01-01 22:00:00    1.39161
2009-01-03 22:00:00    1.39030
2009-01-04 22:00:00    1.36342
2009-01-05 22:00:00    1.35353
                        ...   
2021-10-24 21:00:00    1.16096
2021-10-25 21:00:00    1.15955
2021-10-26 21:00:00    1.16023
2021-10-27 21:00:00    1.16821
2021-10-28 21:00:00    1.15606
Name: bid, Length: 3698, dtype: float64

## Crea un metodo per fare il testing del momentum/contrarian.

In [14]:
# prendero la media del logret degli ultimi "window" giorni come parametro
class IterativeMCBacktest(IterativeBase):
    
    def __init__(self,symbol,start,end,bilancio,fee=(0,0,0)):
        super().__init__(symbol,start,end,bilancio,fee=(0,0,0))
        self.posizione=0 
        
    def go_long(self,t,units=None,amount=None): #chiaramente chiamera la buy
        if self.posizione==-1: #se eh corta debbo prima chiderla per sicurezza
            self.buy(t,units=-self.units)
        if amount is not None:
            if amount=="all":
                amount= self.current_balance
            self.buy(t,amount=amount)
        else: # oko problema
            self.buy(t,units=units)
        self.posizione=1
           

    def go_short(self,t,units=None,amount=None): #chiaramente chiamera la buy
        if self.posizione==1: #se eh corta debbo prima chiderla per sicurezza
            self.sell(t,units=self.units)
        if amount is not None:
            if amount=="all":
                amount= self.current_balance
            self.sell(t,amount=amount)
        else:
            self.sell(t,units=units)
        self.posizione=-1
    
    #do per scontato che ho gia fatto optimize e so gia quale sono i miei paraemtri ottimali 
    def test_MC_strategy(self, isMomentum=True, window=3):
        self.posizione=0
        self.trades=0
        self.current_balance=self.initial_balance
        self.get_data()
        self.data["avglogret"]= self.data.rolling(window).logRet.mean()
        if isMomentum:
            for t in range(len(self.data)-1): 
                if (self.data.avglogret.iloc[t])>0:
                    if self.posizione!=1:
                        self.go_long(t,amount="all")
                        self.posizione=1
                elif (self.data.avglogret.iloc[t])<0:
                    if self.posizione!=-1:
                        self.go_short(t,amount="all")
                        self.posizione=-1  
        else:
            for t in range(len(self.data)-1): 
                if (self.data.avglogret.iloc[t])<0:
                    if self.posizione!=1:
                        self.go_long(t,amount="all")
                        self.posizione=1
                elif (self.data.avglogret.iloc[t])>0:
                    if self.posizione!=-1:
                        self.go_short(t,amount="all") 
                        self.posizione=-1  
        self.close(len(self.data)-1)

In [15]:
bc1 = IterativeMCBacktest("EUR_USD","2018-10-10","2021-10-10",100000)

In [16]:
bc1.test_MC_strategy(isMomentum=True,window=5) 

2018-10-17 21:00:00 Vendo 86955, al prezzo: 1.15002 ricevendo 99999.9891
2018-11-01 21:00:00 Compro 86955, al prezzo: 1.14094 spendendo 99210.43770000001
2018-11-01 21:00:00 Compro 88339, al prezzo: 1.14094 spendendo 100789.49866000001
2018-11-04 22:00:00 Vendo 88339, al prezzo: 1.13853 ricevendo 100576.60167
2018-11-04 22:00:00 Vendo 88339, al prezzo: 1.13853 ricevendo 100576.60167
2018-11-05 22:00:00 Compro 88339, al prezzo: 1.14088 spendendo 100784.19832
2018-11-05 22:00:00 Compro 87975, al prezzo: 1.14088 spendendo 100368.91799999999
2018-11-08 22:00:00 Vendo 87975, al prezzo: 1.13623 ricevendo 99959.83425
2018-11-08 22:00:00 Vendo 87975, al prezzo: 1.13623 ricevendo 99959.83425
2018-11-18 22:00:00 Compro 87975, al prezzo: 1.14162 spendendo 100434.01950000001
2018-11-18 22:00:00 Compro 87144, al prezzo: 1.14162 spendendo 99485.33328
2018-11-25 22:00:00 Vendo 87144, al prezzo: 1.13356 ricevendo 98782.95263999999
2018-11-25 22:00:00 Vendo 87144, al prezzo: 1.13356 ricevendo 98782.952

In [17]:
bc1.data.head(20) #avglogret<0 golong elsewhere goshort

,bid,ask,Abid,Aask,prezzo,logRet,avglogret
time,,,,,,,
2018-10-09 21:00:00,1.15184,1.15235,1.14876,1.14933,1.152095,NaN,NaN
2018-10-10 21:00:00,1.15927,1.15945,1.15189,1.15249,1.159360,0.006286,NaN
2018-10-11 21:00:00,1.15567,1.15627,1.15843,1.15903,1.155970,-0.002928,NaN
2018-10-14 21:00:00,1.15776,1.15801,1.15374,1.15434,1.157885,0.001655,NaN
2018-10-15 21:00:00,1.15734,1.15760,1.15785,1.15805,1.157470,-0.000358,NaN
2018-10-16 21:00:00,1.14991,1.15010,1.15709,1.15758,1.150005,-0.006470,-0.000363
2018-10-17 21:00:00,1.14508,1.14544,1.15002,1.15026,1.145260,-0.004135,-0.002447
2018-10-18 21:00:00,1.15112,1.15172,1.14521,1.14556,1.151420,0.005364,-0.000789
2018-10-21 21:00:00,1.14631,1.14663,1.15141,1.15188,1.146470,-0.004308,-0.001981


In [18]:
bc1.test_MC_strategy(isMomentum=False,window=5)

2018-10-16 21:00:00 Compro 86948, al prezzo: 1.1501 spendendo 99998.8948
2018-10-31 21:00:00 Vendo 86948, al prezzo: 1.14079 ricevendo 99189.40892
2018-10-31 21:00:00 Vendo 86948, al prezzo: 1.14079 ricevendo 99189.40892
2018-11-01 21:00:00 Compro 86948, al prezzo: 1.13905 spendendo 99038.1194
2018-11-01 21:00:00 Compro 87214, al prezzo: 1.13905 spendendo 99341.10669999999
2018-11-04 22:00:00 Vendo 87214, al prezzo: 1.14052 ricevendo 99469.31128
2018-11-04 22:00:00 Vendo 87214, al prezzo: 1.14052 ricevendo 99469.31128
2018-11-07 22:00:00 Compro 87214, al prezzo: 1.13641 spendendo 99110.86174
2018-11-07 22:00:00 Compro 87845, al prezzo: 1.13641 spendendo 99827.93645
2018-11-15 22:00:00 Vendo 87845, al prezzo: 1.1416 ricevendo 100283.852
2018-11-15 22:00:00 Vendo 87845, al prezzo: 1.1416 ricevendo 100283.852
2018-11-22 22:00:00 Compro 87845, al prezzo: 1.13409 spendendo 99624.13605
2018-11-22 22:00:00 Compro 89008, al prezzo: 1.13409 spendendo 100943.08272
2018-12-02 22:00:00 Vendo 89008

In [19]:
bc1.test_MC_strategy(isMomentum=False,window=10)

2018-10-23 21:00:00 Compro 87775, al prezzo: 1.13927 spendendo 99999.42425
2018-11-06 22:00:00 Vendo 87775, al prezzo: 1.14243 ricevendo 100276.79325
2018-11-06 22:00:00 Vendo 87775, al prezzo: 1.14243 ricevendo 100276.79325
2018-11-07 22:00:00 Compro 87775, al prezzo: 1.13641 spendendo 99748.38775
2018-11-07 22:00:00 Compro 88705, al prezzo: 1.13641 spendendo 100805.24905
2018-11-15 22:00:00 Vendo 88705, al prezzo: 1.1416 ricevendo 101265.628
2018-11-15 22:00:00 Vendo 88705, al prezzo: 1.1416 ricevendo 101265.628
2018-11-19 22:00:00 Compro 88705, al prezzo: 1.13721 spendendo 100876.21305
2018-11-19 22:00:00 Compro 89390, al prezzo: 1.13721 spendendo 101655.2019
2018-11-21 22:00:00 Vendo 89390, al prezzo: 1.13985 ricevendo 101891.1915
2018-11-21 22:00:00 Vendo 89390, al prezzo: 1.13985 ricevendo 101891.1915
2018-11-22 22:00:00 Compro 89390, al prezzo: 1.13409 spendendo 101376.3051
2018-11-22 22:00:00 Compro 90298, al prezzo: 1.13409 spendendo 102406.05882
2018-11-25 22:00:00 Vendo 9029

In [20]:
bc1.test_MC_strategy(isMomentum=True,window=10)

2018-10-23 21:00:00 Vendo 87795, al prezzo: 1.13901 ricevendo 99999.38295000001
2018-11-06 22:00:00 Compro 87795, al prezzo: 1.14266 spendendo 100319.8347
2018-11-06 22:00:00 Compro 87234, al prezzo: 1.14266 spendendo 99678.80244
2018-11-07 22:00:00 Vendo 87234, al prezzo: 1.13621 ricevendo 99116.14314
2018-11-07 22:00:00 Vendo 87234, al prezzo: 1.13621 ricevendo 99116.14314
2018-11-15 22:00:00 Compro 87234, al prezzo: 1.14209 spendendo 99629.07906
2018-11-15 22:00:00 Compro 86336, al prezzo: 1.14209 spendendo 98603.48224
2018-11-19 22:00:00 Vendo 86336, al prezzo: 1.13689 ricevendo 98154.53504
2018-11-19 22:00:00 Vendo 86336, al prezzo: 1.13689 ricevendo 98154.53504
2018-11-21 22:00:00 Compro 86336, al prezzo: 1.14041 spendendo 98458.43775999999
2018-11-21 22:00:00 Compro 85803, al prezzo: 1.14041 spendendo 97850.59922999999
2018-11-22 22:00:00 Vendo 85803, al prezzo: 1.13309 ricevendo 97222.52127
2018-11-22 22:00:00 Vendo 85803, al prezzo: 1.13309 ricevendo 97222.52127
2018-11-25 22:

In [21]:
bc1.test_MC_strategy(isMomentum=True,window=100)

2019-02-28 22:00:00 Vendo 88024, al prezzo: 1.13605 ricevendo 99999.6652
2019-03-19 21:00:00 Compro 88024, al prezzo: 1.14138 spendendo 100468.83312000001
2019-03-19 21:00:00 Compro 87202, al prezzo: 1.14138 spendendo 99530.61876000001
2019-03-21 21:00:00 Vendo 87202, al prezzo: 1.12981 ricevendo 98521.69162
2019-03-21 21:00:00 Vendo 87202, al prezzo: 1.12981 ricevendo 98521.69162
2019-04-02 21:00:00 Compro 87202, al prezzo: 1.12342 spendendo 97964.47084000001
2019-04-02 21:00:00 Compro 88194, al prezzo: 1.12342 spendendo 99078.90348000001
2019-04-03 21:00:00 Vendo 88194, al prezzo: 1.12192 ricevendo 98946.61248
2019-04-03 21:00:00 Vendo 88194, al prezzo: 1.12192 ricevendo 98946.61248
2019-06-25 21:00:00 Compro 88194, al prezzo: 1.13736 spendendo 100308.32784
2019-06-25 21:00:00 Compro 85799, al prezzo: 1.13736 spendendo 97584.35063999999
2019-07-01 21:00:00 Vendo 85799, al prezzo: 1.12849 ricevendo 96823.31350999999
2019-07-01 21:00:00 Vendo 85799, al prezzo: 1.12849 ricevendo 96823.3

In [22]:
bc1.test_MC_strategy(isMomentum=False,window=100)

2019-02-28 22:00:00 Compro 87956, al prezzo: 1.13692 spendendo 99998.93552
2019-03-19 21:00:00 Vendo 87956, al prezzo: 1.14117 ricevendo 100372.74852000001
2019-03-19 21:00:00 Vendo 87956, al prezzo: 1.14117 ricevendo 100372.74852000001
2019-03-21 21:00:00 Compro 87956, al prezzo: 1.13046 spendendo 99430.73976
2019-03-21 21:00:00 Compro 89623, al prezzo: 1.13046 spendendo 101315.21658000001
2019-04-02 21:00:00 Vendo 89623, al prezzo: 1.12318 ricevendo 100662.76114
2019-04-02 21:00:00 Vendo 89623, al prezzo: 1.12318 ricevendo 100662.76114
2019-04-03 21:00:00 Compro 89623, al prezzo: 1.12219 spendendo 100574.03437000001
2019-04-03 21:00:00 Compro 89781, al prezzo: 1.12219 spendendo 100751.34039
2019-06-25 21:00:00 Vendo 89781, al prezzo: 1.13636 ricevendo 102023.53716
2019-06-25 21:00:00 Vendo 89781, al prezzo: 1.13636 ricevendo 102023.53716
2019-07-01 21:00:00 Compro 89781, al prezzo: 1.12871 spendendo 101336.71251000001
2019-07-01 21:00:00 Compro 90998, al prezzo: 1.12871 spendendo 102

In [23]:
bc1.test_MC_strategy(isMomentum=False,window=2)

2018-10-11 21:00:00 Vendo 86529, al prezzo: 1.15567 ricevendo 99998.96943
2018-10-14 21:00:00 Compro 86529, al prezzo: 1.15801 spendendo 100201.44729
2018-10-14 21:00:00 Compro 86180, al prezzo: 1.15801 spendendo 99797.3018
2018-10-15 21:00:00 Vendo 86180, al prezzo: 1.15734 ricevendo 99739.5612
2018-10-15 21:00:00 Vendo 86180, al prezzo: 1.15734 ricevendo 99739.5612
2018-10-16 21:00:00 Compro 86180, al prezzo: 1.1501 spendendo 99115.61799999999
2018-10-16 21:00:00 Compro 87265, al prezzo: 1.1501 spendendo 100363.47649999999
2018-10-18 21:00:00 Vendo 87265, al prezzo: 1.15112 ricevendo 100452.4868
2018-10-18 21:00:00 Vendo 87265, al prezzo: 1.15112 ricevendo 100452.4868
2018-10-22 21:00:00 Compro 87265, al prezzo: 1.14745 spendendo 100132.22425000001
2018-10-22 21:00:00 Compro 87823, al prezzo: 1.14745 spendendo 100772.50135
2018-10-25 21:00:00 Vendo 87823, al prezzo: 1.14003 ricevendo 100120.85469000001
2018-10-25 21:00:00 Vendo 87823, al prezzo: 1.14003 ricevendo 100120.85469000001
2

In [24]:
bc1.test_MC_strategy(isMomentum=True,window=2)

2018-10-11 21:00:00 Compro 86484, al prezzo: 1.15627 spendendo 99998.85467999999
2018-10-14 21:00:00 Vendo 86484, al prezzo: 1.15776 ricevendo 100127.71583999999
2018-10-14 21:00:00 Vendo 86484, al prezzo: 1.15776 ricevendo 100127.71583999999
2018-10-15 21:00:00 Compro 86484, al prezzo: 1.1576 spendendo 100113.8784
2018-10-15 21:00:00 Compro 86508, al prezzo: 1.1576 spendendo 100141.6608
2018-10-16 21:00:00 Vendo 86508, al prezzo: 1.14991 ricevendo 99476.41428
2018-10-16 21:00:00 Vendo 86508, al prezzo: 1.14991 ricevendo 99476.41428
2018-10-18 21:00:00 Compro 86508, al prezzo: 1.15172 spendendo 99632.99376000001
2018-10-18 21:00:00 Compro 86236, al prezzo: 1.15172 spendendo 99319.72592000001
2018-10-22 21:00:00 Vendo 86236, al prezzo: 1.14697 ricevendo 98910.10492
2018-10-22 21:00:00 Vendo 86236, al prezzo: 1.14697 ricevendo 98910.10492
2018-10-25 21:00:00 Compro 86236, al prezzo: 1.14063 spendendo 98363.36868
2018-10-25 21:00:00 Compro 87195, al prezzo: 1.14063 spendendo 99457.23285
2

In [25]:
bc1.test_MC_strategy(isMomentum=True,window=5)

2018-10-16 21:00:00 Vendo 86963, al prezzo: 1.14991 ricevendo 99999.62333
2018-10-31 21:00:00 Compro 86963, al prezzo: 1.14098 spendendo 99223.04374000001
2018-10-31 21:00:00 Compro 88324, al prezzo: 1.14098 spendendo 100775.91752
2018-11-01 21:00:00 Vendo 88324, al prezzo: 1.13845 ricevendo 100552.4578
2018-11-01 21:00:00 Vendo 88324, al prezzo: 1.13845 ricevendo 100552.4578
2018-11-04 22:00:00 Compro 88324, al prezzo: 1.14081 spendendo 100760.90244
2018-11-04 22:00:00 Compro 87959, al prezzo: 1.14081 spendendo 100344.50679000001
2018-11-07 22:00:00 Vendo 87959, al prezzo: 1.13621 ricevendo 99939.89538999999
2018-11-07 22:00:00 Vendo 87959, al prezzo: 1.13621 ricevendo 99939.89538999999
2018-11-15 22:00:00 Compro 87959, al prezzo: 1.14209 spendendo 100457.09431
2018-11-15 22:00:00 Compro 87053, al prezzo: 1.14209 spendendo 99422.36077
2018-11-22 22:00:00 Vendo 87053, al prezzo: 1.13309 ricevendo 98638.88377
2018-11-22 22:00:00 Vendo 87053, al prezzo: 1.13309 ricevendo 98638.88377
2018

In [26]:
bc1.test_MC_strategy(isMomentum=False,window=5)

2018-10-16 21:00:00 Compro 86948, al prezzo: 1.1501 spendendo 99998.8948
2018-10-31 21:00:00 Vendo 86948, al prezzo: 1.14079 ricevendo 99189.40892
2018-10-31 21:00:00 Vendo 86948, al prezzo: 1.14079 ricevendo 99189.40892
2018-11-01 21:00:00 Compro 86948, al prezzo: 1.13905 spendendo 99038.1194
2018-11-01 21:00:00 Compro 87214, al prezzo: 1.13905 spendendo 99341.10669999999
2018-11-04 22:00:00 Vendo 87214, al prezzo: 1.14052 ricevendo 99469.31128
2018-11-04 22:00:00 Vendo 87214, al prezzo: 1.14052 ricevendo 99469.31128
2018-11-07 22:00:00 Compro 87214, al prezzo: 1.13641 spendendo 99110.86174
2018-11-07 22:00:00 Compro 87845, al prezzo: 1.13641 spendendo 99827.93645
2018-11-15 22:00:00 Vendo 87845, al prezzo: 1.1416 ricevendo 100283.852
2018-11-15 22:00:00 Vendo 87845, al prezzo: 1.1416 ricevendo 100283.852
2018-11-22 22:00:00 Compro 87845, al prezzo: 1.13409 spendendo 99624.13605
2018-11-22 22:00:00 Compro 89008, al prezzo: 1.13409 spendendo 100943.08272
2018-12-02 22:00:00 Vendo 89008

In [27]:
bc1.data.head(20)

,bid,ask,Abid,Aask,prezzo,logRet,avglogret
time,,,,,,,
2018-10-09 21:00:00,1.15184,1.15235,1.14876,1.14933,1.152095,NaN,NaN
2018-10-10 21:00:00,1.15927,1.15945,1.15189,1.15249,1.159360,0.006286,NaN
2018-10-11 21:00:00,1.15567,1.15627,1.15843,1.15903,1.155970,-0.002928,NaN
2018-10-14 21:00:00,1.15776,1.15801,1.15374,1.15434,1.157885,0.001655,NaN
2018-10-15 21:00:00,1.15734,1.15760,1.15785,1.15805,1.157470,-0.000358,NaN
2018-10-16 21:00:00,1.14991,1.15010,1.15709,1.15758,1.150005,-0.006470,-0.000363
2018-10-17 21:00:00,1.14508,1.14544,1.15002,1.15026,1.145260,-0.004135,-0.002447
2018-10-18 21:00:00,1.15112,1.15172,1.14521,1.14556,1.151420,0.005364,-0.000789
2018-10-21 21:00:00,1.14631,1.14663,1.15141,1.15188,1.146470,-0.004308,-0.001981


In [28]:
df1=bc1.data
df1

,bid,ask,Abid,Aask,prezzo,logRet,avglogret
time,,,,,,,
2018-10-09 21:00:00,1.15184,1.15235,1.14876,1.14933,1.152095,NaN,NaN
2018-10-10 21:00:00,1.15927,1.15945,1.15189,1.15249,1.159360,0.006286,NaN
2018-10-11 21:00:00,1.15567,1.15627,1.15843,1.15903,1.155970,-0.002928,NaN
2018-10-14 21:00:00,1.15776,1.15801,1.15374,1.15434,1.157885,0.001655,NaN
2018-10-15 21:00:00,1.15734,1.15760,1.15785,1.15805,1.157470,-0.000358,NaN
...,...,...,...,...,...,...,...
2021-10-03 21:00:00,1.16174,1.16233,1.15930,1.16030,1.162035,0.002356,-0.001286
2021-10-04 21:00:00,1.15963,1.15994,1.16170,1.16228,1.159785,-0.001938,-0.001470
2021-10-05 21:00:00,1.15548,1.15586,1.15968,1.16007,1.155670,-0.003554,-0.000698


## Crea un metodo per fare il testing delle bande di Bollinger.

In [29]:
# Chiaramente, la banda de bollinger none la strategia piu adatta per questo tipo di 
# handling dei dati finanziari. 


class IterativeBollingerBacktest(IterativeBase):
    
    def __init__(self,symbol,start,end,bilancio,fee=(0,0,0)):
        super().__init__(symbol,start,end,bilancio,fee=(0,0,0))
        self.posizione=0 

        
    def go_long(self,t,units=None,amount=None): #chiaramente chiamera la buy
        if self.posizione==-1: #se eh corta debbo prima chiderla per sicurezza
            self.buy(t,units=-self.units)
        if amount is not None:
            if amount=="all":
                amount= self.current_balance
            self.buy(t,amount=amount)
            
        else: # oko problema
            self.buy(t,units=units)
        self.posizione=1
           

    def go_short(self,t,units=None,amount=None): #chiaramente chiamera la buy
        if self.posizione==1: #se eh corta debbo prima chiderla per sicurezza
            self.sell(t,units=self.units)
        if amount is not None:
            if amount=="all":
                amount= self.current_balance
            self.sell(t,amount=amount)
        else:
            self.sell(t,units=units)
        self.posizione=-1
    
    #do per scontato che ho gia fatto optimize e so gia quale sono i miei paraemtri ottimali 
    def test_bollinger_strategy(self, numdevst=2, window=3):
        self.posizione=0
        self.trades=0
        self.current_balance=self.initial_balance
        self.get_data()
        self.data["SMAAsk"]=self.data.ask.rolling(window).mean() # oko shiftare?
        self.data["SMABid"]=self.data.bid.rolling(window).mean()
        self.data["devAsk"]=self.data.ask.rolling(window).std()
        self.data["devBid"]=self.data.bid.rolling(window).std()
        self.data["lower"]=self.data.SMAAsk-(numdevst*self.data.devAsk)
        self.data["upper"]=self.data.SMABid+(numdevst*self.data.devBid)
       #se faccio dropna mi vengono errori 
        for t in range(len(self.data)-1): 
            if (self.data.iloc[t].ask < self.data.iloc[t].lower): #aprire pos. lunga
                if self.posizione!=1:
                    self.go_long(t,amount="all")
                    self.posizione=1      
            elif (self.data.iloc[t].bid > self.data.iloc[t].upper): #corta
                if self.posizione!=-1:
                    self.go_short(t,amount="all")
                    self.posizione=-1 
        self.close(len(self.data)-1)

In [30]:
cols=["ask","lower","Aask","bid","upper","Abid"]

In [31]:
BBtest1 = IterativeBollingerBacktest("EUR_USD","2018-10-10","2021-10-10",100000)

In [32]:
BBtest1.print_current_situation(10)

2018-10-23 21:00:00 INVESTITI: 0 per un totale di 0.0


In [33]:
BBtest1.sell(64,1000)

2019-01-10 22:00:00 Vendo 1000, al prezzo: 1.14984 ricevendo 1149.84


In [34]:
#commando per confermare che ho venduto il giorno 64 per il prezzo Abid prezzo di apertura Bid del giorno prossimo 65
BBtest1.data.iloc[65]

bid       1.146370
ask       1.147370
Abid      1.149840
Aask      1.150080
prezzo    1.146870
logRet   -0.002747
Name: 2019-01-10 22:00:00, dtype: float64

In [35]:
BBtest1.print_current_situation(65)

2019-01-10 22:00:00 INVESTITI: -1000 per un totale di -1150.08


In [36]:
BBtest1.test_bollinger_strategy(0.333,2)

2018-10-11 21:00:00 Vendo 86323, al prezzo: 1.15843 ricevendo 99999.15289000001
2018-10-14 21:00:00 Compro 87323, al prezzo: 1.15434 spendendo 100800.43182
2018-10-14 21:00:00 Compro 85935, al prezzo: 1.15434 spendendo 99198.2079
2018-10-15 21:00:00 Vendo 85935, al prezzo: 1.15785 ricevendo 99499.83975
2018-10-15 21:00:00 Vendo 85935, al prezzo: 1.15785 ricevendo 99499.83975
2018-10-16 21:00:00 Compro 85935, al prezzo: 1.15758 spendendo 99476.6373
2018-10-16 21:00:00 Compro 85975, al prezzo: 1.15758 spendendo 99522.94050000001
2018-10-21 21:00:00 Vendo 85975, al prezzo: 1.15141 ricevendo 98992.47475000001
2018-10-21 21:00:00 Vendo 85975, al prezzo: 1.15141 ricevendo 98992.47475000001
2018-10-22 21:00:00 Compro 85975, al prezzo: 1.14663 spendendo 98581.51425000001
2018-10-22 21:00:00 Compro 86692, al prezzo: 1.14663 spendendo 99403.64796
2018-10-23 21:00:00 Vendo 86692, al prezzo: 1.14704 ricevendo 99439.19168
2018-10-23 21:00:00 Vendo 86692, al prezzo: 1.14704 ricevendo 99439.19168
201

2019-08-25 21:00:00 Vendo 85285, al prezzo: 1.11497 ricevendo 95090.21645
2019-08-25 21:00:00 Vendo 85285, al prezzo: 1.11497 ricevendo 95090.21645
2019-08-26 21:00:00 Compro 85285, al prezzo: 1.11065 spendendo 94721.78524999999
2019-08-26 21:00:00 Compro 85948, al prezzo: 1.11065 spendendo 95458.14619999999
2019-09-03 21:00:00 Vendo 85948, al prezzo: 1.09717 ricevendo 94299.56715999999
2019-09-03 21:00:00 Vendo 85948, al prezzo: 1.09717 ricevendo 94299.56715999999
2019-09-05 21:00:00 Compro 85948, al prezzo: 1.10357 spendendo 94849.63436
2019-09-05 21:00:00 Compro 84952, al prezzo: 1.10357 spendendo 93750.47864
2019-09-09 21:00:00 Vendo 84952, al prezzo: 1.10449 ricevendo 93828.63448
2019-09-09 21:00:00 Vendo 84952, al prezzo: 1.10449 ricevendo 93828.63448
2019-09-10 21:00:00 Compro 84952, al prezzo: 1.1046 spendendo 93837.9792
2019-09-10 21:00:00 Compro 84935, al prezzo: 1.1046 spendendo 93819.201
2019-09-12 21:00:00 Vendo 84935, al prezzo: 1.10628 ricevendo 93961.8918
2019-09-12 21:

2020-06-07 21:00:00 Compro 77288, al prezzo: 1.13032 spendendo 87360.17216
2020-06-07 21:00:00 Compro 72880, al prezzo: 1.13032 spendendo 82377.7216
2020-06-08 21:00:00 Vendo 72880, al prezzo: 1.12909 ricevendo 82288.0792
2020-06-08 21:00:00 Vendo 72880, al prezzo: 1.12909 ricevendo 82288.0792
2020-06-11 21:00:00 Compro 72880, al prezzo: 1.13006 spendendo 82358.7728
2020-06-11 21:00:00 Compro 72755, al prezzo: 1.13006 spendendo 82217.5153
2020-06-15 21:00:00 Vendo 72755, al prezzo: 1.13226 ricevendo 82377.5763
2020-06-15 21:00:00 Vendo 72755, al prezzo: 1.13226 ricevendo 82377.5763
2020-06-16 21:00:00 Compro 72755, al prezzo: 1.12686 spendendo 81984.6993
2020-06-16 21:00:00 Compro 73452, al prezzo: 1.12686 spendendo 82770.12071999999
2020-06-22 21:00:00 Vendo 73452, al prezzo: 1.12575 ricevendo 82688.589
2020-06-22 21:00:00 Vendo 73452, al prezzo: 1.12575 ricevendo 82688.589
2020-06-24 21:00:00 Compro 73452, al prezzo: 1.12567 spendendo 82682.71284
2020-06-24 21:00:00 Compro 73463, al 

2021-03-22 21:00:00 Vendo 64852, al prezzo: 1.19304 ricevendo 77371.03008000001
2021-03-22 21:00:00 Vendo 64852, al prezzo: 1.19304 ricevendo 77371.03008000001
2021-03-23 21:00:00 Compro 64852, al prezzo: 1.1851 spendendo 76856.1052
2021-03-23 21:00:00 Compro 65721, al prezzo: 1.1851 spendendo 77885.9571
2021-03-28 21:00:00 Vendo 65721, al prezzo: 1.17926 ricevendo 77502.14646
2021-03-28 21:00:00 Vendo 65721, al prezzo: 1.17926 ricevendo 77502.14646
2021-03-29 21:00:00 Compro 65721, al prezzo: 1.17651 spendendo 77321.41371
2021-03-29 21:00:00 Compro 66028, al prezzo: 1.17651 spendendo 77682.60227999999
2021-03-31 21:00:00 Vendo 66028, al prezzo: 1.17282 ricevendo 77438.95896
2021-03-31 21:00:00 Vendo 66028, al prezzo: 1.17282 ricevendo 77438.95896
2021-04-04 21:00:00 Compro 66028, al prezzo: 1.17658 spendendo 77687.22424
2021-04-04 21:00:00 Compro 65606, al prezzo: 1.17658 spendendo 77190.70748
2021-04-05 21:00:00 Vendo 65606, al prezzo: 1.18107 ricevendo 77485.27842
2021-04-05 21:00:0

In [37]:
# dalla cellula sopra si vede che questa strategia non e efficiente

In [38]:
BBtest1.data[cols].head(20) # if ask<lower go_long or bid>upper go_short

,ask,lower,Aask,bid,upper,Abid
time,,,,,,
2018-10-09 21:00:00,1.15235,NaN,1.14933,1.15184,NaN,1.14876
2018-10-10 21:00:00,1.15945,1.154228,1.15249,1.15927,1.157305,1.15189
2018-10-11 21:00:00,1.15627,1.157111,1.15903,1.15567,1.158318,1.15843
2018-10-14 21:00:00,1.15801,1.156730,1.15434,1.15776,1.157207,1.15374
2018-10-15 21:00:00,1.15760,1.157708,1.15805,1.15734,1.157649,1.15785
2018-10-16 21:00:00,1.15010,1.152084,1.15758,1.14991,1.155375,1.15709
2018-10-17 21:00:00,1.14544,1.146673,1.15026,1.14508,1.148632,1.15002
2018-10-18 21:00:00,1.15172,1.147101,1.14556,1.15112,1.149522,1.14521
2018-10-21 21:00:00,1.14663,1.147976,1.15188,1.14631,1.149848,1.15141


In [39]:
BBtest1.test_bollinger_strategy(0.7,2)

2018-10-10 21:00:00 Vendo 86261, al prezzo: 1.15927 ricevendo 99999.78947
2018-10-11 21:00:00 Compro 86261, al prezzo: 1.15627 spendendo 99741.00647
2018-10-11 21:00:00 Compro 86708, al prezzo: 1.15627 spendendo 100257.85915999999
2018-10-14 21:00:00 Vendo 86708, al prezzo: 1.15776 ricevendo 100387.05407999999
2018-10-14 21:00:00 Vendo 86708, al prezzo: 1.15776 ricevendo 100387.05407999999
2018-10-15 21:00:00 Compro 86708, al prezzo: 1.1576 spendendo 100373.1808
2018-10-15 21:00:00 Compro 86732, al prezzo: 1.1576 spendendo 100400.9632
2018-10-18 21:00:00 Vendo 86732, al prezzo: 1.15112 ricevendo 99838.93983999999
2018-10-18 21:00:00 Vendo 86732, al prezzo: 1.15112 ricevendo 99838.93983999999
2018-10-21 21:00:00 Compro 86732, al prezzo: 1.14663 spendendo 99449.51316
2018-10-21 21:00:00 Compro 87412, al prezzo: 1.14663 spendendo 100229.22156
2018-10-22 21:00:00 Vendo 87412, al prezzo: 1.14697 ricevendo 100258.94164
2018-10-22 21:00:00 Vendo 87412, al prezzo: 1.14697 ricevendo 100258.9416

2019-05-20 21:00:00 Compro 90337, al prezzo: 1.11639 spendendo 100851.32343
2019-05-20 21:00:00 Compro 90343, al prezzo: 1.11639 spendendo 100858.02177
2019-05-22 21:00:00 Vendo 90343, al prezzo: 1.11807 ricevendo 101009.79800999998
2019-05-22 21:00:00 Vendo 90343, al prezzo: 1.11807 ricevendo 101009.79800999998
2019-05-26 21:00:00 Compro 90343, al prezzo: 1.1198 spendendo 101166.09139999999
2019-05-26 21:00:00 Compro 90064, al prezzo: 1.1198 spendendo 100853.6672
2019-05-30 21:00:00 Vendo 90064, al prezzo: 1.11675 ricevendo 100578.972
2019-05-30 21:00:00 Vendo 90064, al prezzo: 1.11675 ricevendo 100578.972
2019-06-04 21:00:00 Compro 90064, al prezzo: 1.12225 spendendo 101074.324
2019-06-04 21:00:00 Compro 89181, al prezzo: 1.12225 spendendo 100083.37724999999
2019-06-05 21:00:00 Vendo 89181, al prezzo: 1.12736 ricevendo 100539.09215999999
2019-06-05 21:00:00 Vendo 89181, al prezzo: 1.12736 ricevendo 100539.09215999999
2019-06-09 21:00:00 Compro 89181, al prezzo: 1.13126 spendendo 1008

2020-02-10 22:00:00 Vendo 86181, al prezzo: 1.09146 ricevendo 94063.11426
2020-02-10 22:00:00 Vendo 86181, al prezzo: 1.09146 ricevendo 94063.11426
2020-02-11 22:00:00 Compro 86181, al prezzo: 1.08748 spendendo 93720.11388
2020-02-11 22:00:00 Compro 86812, al prezzo: 1.08748 spendendo 94406.31376
2020-02-16 22:00:00 Vendo 86812, al prezzo: 1.08312 ricevendo 94027.81344000001
2020-02-16 22:00:00 Vendo 86812, al prezzo: 1.08312 ricevendo 94027.81344000001
2020-02-17 22:00:00 Compro 86812, al prezzo: 1.07927 spendendo 93693.58724
2020-02-17 22:00:00 Compro 87431, al prezzo: 1.07927 spendendo 94361.65537
2020-02-18 22:00:00 Vendo 87431, al prezzo: 1.08047 ricevendo 94466.57257
2020-02-18 22:00:00 Vendo 87431, al prezzo: 1.08047 ricevendo 94466.57257
2020-02-19 22:00:00 Compro 87431, al prezzo: 1.07866 spendendo 94308.32246
2020-02-19 22:00:00 Compro 87725, al prezzo: 1.07866 spendendo 94625.4485
2020-02-20 22:00:00 Vendo 87725, al prezzo: 1.08459 ricevendo 95145.65775
2020-02-20 22:00:00 V

2020-09-27 21:00:00 Vendo 72098, al prezzo: 1.16591 ricevendo 84059.77918
2020-09-27 21:00:00 Vendo 72098, al prezzo: 1.16591 ricevendo 84059.77918
2020-09-29 21:00:00 Compro 72098, al prezzo: 1.17229 spendendo 84519.76442
2020-09-29 21:00:00 Compro 71313, al prezzo: 1.17229 spendendo 83599.51677
2020-09-30 21:00:00 Vendo 71313, al prezzo: 1.17451 ricevendo 83757.83163
2020-09-30 21:00:00 Vendo 71313, al prezzo: 1.17451 ricevendo 83757.83163
2020-10-01 21:00:00 Compro 71313, al prezzo: 1.17181 spendendo 83565.28653
2020-10-01 21:00:00 Compro 71642, al prezzo: 1.17181 spendendo 83950.81202
2020-10-04 21:00:00 Vendo 71642, al prezzo: 1.17838 ricevendo 84421.49996
2020-10-04 21:00:00 Vendo 71642, al prezzo: 1.17838 ricevendo 84421.49996
2020-10-05 21:00:00 Compro 71642, al prezzo: 1.17352 spendendo 84073.31984
2020-10-05 21:00:00 Compro 72235, al prezzo: 1.17352 spendendo 84769.2172
2020-10-06 21:00:00 Vendo 72235, al prezzo: 1.17611 ricevendo 84956.30585
2020-10-06 21:00:00 Vendo 72235, 

2021-06-23 21:00:00 Vendo 71047, al prezzo: 1.19305 ricevendo 84762.62335
2021-06-27 21:00:00 Compro 71047, al prezzo: 1.19251 spendendo 84724.25796999999
2021-06-27 21:00:00 Compro 71111, al prezzo: 1.19251 spendendo 84800.57861
2021-07-01 21:00:00 Vendo 71111, al prezzo: 1.18643 ricevendo 84368.22373000001
2021-07-01 21:00:00 Vendo 71111, al prezzo: 1.18643 ricevendo 84368.22373000001
2021-07-05 21:00:00 Compro 71111, al prezzo: 1.18239 spendendo 84080.93529000001
2021-07-05 21:00:00 Compro 71597, al prezzo: 1.18239 spendendo 84655.57683
2021-07-07 21:00:00 Vendo 71597, al prezzo: 1.18452 ricevendo 84808.07844
2021-07-07 21:00:00 Vendo 71597, al prezzo: 1.18452 ricevendo 84808.07844
2021-07-11 21:00:00 Compro 71597, al prezzo: 1.18621 spendendo 84929.07737
2021-07-11 21:00:00 Compro 71393, al prezzo: 1.18621 spendendo 84687.09053
2021-07-13 21:00:00 Vendo 71393, al prezzo: 1.18355 ricevendo 84497.18515
2021-07-13 21:00:00 Vendo 71393, al prezzo: 1.18355 ricevendo 84497.18515
2021-07-

In [40]:
BBtest1.test_bollinger_strategy(0.1,3)

2018-10-14 21:00:00 Vendo 86373, al prezzo: 1.15776 ricevendo 99999.20447999999
2018-10-16 21:00:00 Compro 86373, al prezzo: 1.1501 spendendo 99337.58729999998
2018-10-16 21:00:00 Compro 87524, al prezzo: 1.1501 spendendo 100661.35239999999
2018-10-18 21:00:00 Vendo 87524, al prezzo: 1.15112 ricevendo 100750.62688
2018-10-18 21:00:00 Vendo 87524, al prezzo: 1.15112 ricevendo 100750.62688
2018-10-21 21:00:00 Compro 87524, al prezzo: 1.14663 spendendo 100357.64412
2018-10-21 21:00:00 Compro 88209, al prezzo: 1.14663 spendendo 101143.08567
2018-10-25 21:00:00 Vendo 88209, al prezzo: 1.14003 ricevendo 100560.90627
2018-10-25 21:00:00 Vendo 88209, al prezzo: 1.14003 ricevendo 100560.90627
2018-10-28 21:00:00 Compro 88209, al prezzo: 1.13743 spendendo 100331.56287
2018-10-28 21:00:00 Compro 88612, al prezzo: 1.13743 spendendo 100789.94716
2018-10-31 21:00:00 Vendo 88612, al prezzo: 1.14079 ricevendo 101087.68347999999
2018-10-31 21:00:00 Vendo 88612, al prezzo: 1.14079 ricevendo 101087.68347

2020-03-04 22:00:00 Vendo 84418, al prezzo: 1.12355 ricevendo 94847.8439
2020-03-09 21:00:00 Compro 84418, al prezzo: 1.12838 spendendo 95255.58283999999
2020-03-09 21:00:00 Compro 83695, al prezzo: 1.12838 spendendo 94439.7641
2020-03-15 21:00:00 Vendo 83695, al prezzo: 1.1179 ricevendo 93562.6405
2020-03-15 21:00:00 Vendo 83695, al prezzo: 1.1179 ricevendo 93562.6405
2020-03-16 21:00:00 Compro 83695, al prezzo: 1.09984 spendendo 92051.10879999999
2020-03-16 21:00:00 Compro 86444, al prezzo: 1.09984 spendendo 95074.56895999999
2020-03-22 21:00:00 Vendo 86444, al prezzo: 1.07218 ricevendo 92683.52792
2020-03-22 21:00:00 Vendo 86444, al prezzo: 1.07218 ricevendo 92683.52792
2020-03-29 21:00:00 Compro 86444, al prezzo: 1.10503 spendendo 95523.21332
2020-03-29 21:00:00 Compro 81304, al prezzo: 1.10503 spendendo 89843.35912
2020-04-06 21:00:00 Vendo 81304, al prezzo: 1.08905 ricevendo 88544.12120000001
2020-04-06 21:00:00 Vendo 81304, al prezzo: 1.08905 ricevendo 88544.12120000001
2020-04-

2020-12-20 22:00:00 Compro 70267, al prezzo: 1.22464 spendendo 86051.77888
2020-12-20 22:00:00 Compro 69055, al prezzo: 1.22464 spendendo 84567.5152
2020-12-23 22:00:00 Vendo 69055, al prezzo: 1.21828 ricevendo 84128.3254
2020-12-23 22:00:00 Vendo 69055, al prezzo: 1.21828 ricevendo 84128.3254
2020-12-30 22:00:00 Compro 69055, al prezzo: 1.2219 spendendo 84378.3045
2020-12-30 22:00:00 Compro 68646, al prezzo: 1.2219 spendendo 83878.5474
2021-01-04 22:00:00 Vendo 68646, al prezzo: 1.22969 ricevendo 84413.29974
2021-01-04 22:00:00 Vendo 68646, al prezzo: 1.22969 ricevendo 84413.29974
2021-01-06 22:00:00 Compro 68646, al prezzo: 1.22706 spendendo 84232.76076
2021-01-06 22:00:00 Compro 68940, al prezzo: 1.22706 spendendo 84593.51640000001
2021-01-11 22:00:00 Vendo 68940, al prezzo: 1.22069 ricevendo 84154.3686
2021-01-11 22:00:00 Vendo 68940, al prezzo: 1.22069 ricevendo 84154.3686
2021-01-12 22:00:00 Compro 68940, al prezzo: 1.21589 spendendo 83823.45659999999
2021-01-12 22:00:00 Compro 6

In [41]:
BBtest1.data[cols]

,ask,lower,Aask,bid,upper,Abid
time,,,,,,
2018-10-09 21:00:00,1.15235,NaN,1.14933,1.15184,NaN,1.14876
2018-10-10 21:00:00,1.15945,NaN,1.15249,1.15927,NaN,1.15189
2018-10-11 21:00:00,1.15627,1.155668,1.15903,1.15567,1.155965,1.15843
2018-10-14 21:00:00,1.15801,1.157751,1.15434,1.15776,1.157747,1.15374
2018-10-15 21:00:00,1.15760,1.157202,1.15805,1.15734,1.157034,1.15785
...,...,...,...,...,...,...
2021-10-03 21:00:00,1.16233,1.159654,1.16030,1.16174,1.159721,1.15930
2021-10-04 21:00:00,1.15994,1.160408,1.16228,1.15963,1.160320,1.16170
2021-10-05 21:00:00,1.15586,1.159050,1.16007,1.15548,1.159268,1.15968


In [42]:
BBtest1.test_bollinger_strategy(2,1)

Non ce posizione da chiudere!
------CHIUSO TUTO--------
2021-10-07 21:00:00 BILANCIO: 100000 PERFORMANCE: 0.0%, TRADE TOTALI: 0


In [43]:
BBtest1.test_bollinger_strategy(13,25)

Non ce posizione da chiudere!
------CHIUSO TUTO--------
2021-10-07 21:00:00 BILANCIO: 100000 PERFORMANCE: 0.0%, TRADE TOTALI: 0


In [44]:
BBtest1.test_bollinger_strategy(1.2,3)

Non ce posizione da chiudere!
------CHIUSO TUTO--------
2021-10-07 21:00:00 BILANCIO: 100000 PERFORMANCE: 0.0%, TRADE TOTALI: 0


In [45]:
BBtest1.data[cols].tail()

,ask,lower,Aask,bid,upper,Abid
time,,,,,,
2021-10-03 21:00:00,1.16233,1.157173,1.16030,1.16174,1.162009,1.15930
2021-10-04 21:00:00,1.15994,1.158701,1.16228,1.15963,1.161823,1.16170
2021-10-05 21:00:00,1.15586,1.155451,1.16007,1.15548,1.162772,1.15968
2021-10-06 21:00:00,1.15545,1.154104,1.15607,1.15528,1.159744,1.15541
2021-10-07 21:00:00,1.15784,1.154850,1.15563,1.15704,1.157090,1.15517


In [46]:
BBtest1.test_bollinger_strategy(22,44)

Non ce posizione da chiudere!
------CHIUSO TUTO--------
2021-10-07 21:00:00 BILANCIO: 100000 PERFORMANCE: 0.0%, TRADE TOTALI: 0


In [47]:
BBtest1.test_bollinger_strategy(44,22)

Non ce posizione da chiudere!
------CHIUSO TUTO--------
2021-10-07 21:00:00 BILANCIO: 100000 PERFORMANCE: 0.0%, TRADE TOTALI: 0


In [48]:
BBtest1.test_bollinger_strategy(15,65)

Non ce posizione da chiudere!
------CHIUSO TUTO--------
2021-10-07 21:00:00 BILANCIO: 100000 PERFORMANCE: 0.0%, TRADE TOTALI: 0


In [49]:
BBtest1.data

,bid,ask,Abid,Aask,prezzo,logRet,SMAAsk,SMABid,devAsk,devBid,lower,upper
time,,,,,,,,,,,,
2018-10-09 21:00:00,1.15184,1.15235,1.14876,1.14933,1.152095,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-10 21:00:00,1.15927,1.15945,1.15189,1.15249,1.159360,0.006286,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-11 21:00:00,1.15567,1.15627,1.15843,1.15903,1.155970,-0.002928,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-14 21:00:00,1.15776,1.15801,1.15374,1.15434,1.157885,0.001655,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-15 21:00:00,1.15734,1.15760,1.15785,1.15805,1.157470,-0.000358,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-03 21:00:00,1.16174,1.16233,1.15930,1.16030,1.162035,0.002356,1.177789,1.177575,0.007185,0.007186,1.070013,1.285360
2021-10-04 21:00:00,1.15963,1.15994,1.16170,1.16228,1.159785,-0.001938,1.177443,1.177228,0.007493,0.007497,1.065043,1.289681
2021-10-05 21:00:00,1.15548,1.15586,1.15968,1.16007,1.155670,-0.003554,1.177085,1.176865,0.007953,0.007963,1.057789,1.296310


In [50]:
BBtest1.test_bollinger_strategy(65,15)

Non ce posizione da chiudere!
------CHIUSO TUTO--------
2021-10-07 21:00:00 BILANCIO: 100000 PERFORMANCE: 0.0%, TRADE TOTALI: 0


In [51]:
# dalla lettura tutte queste tabelline sopra si vede e si conferma che sotto questa strategia 
# e considerando una deviazione standard uguale a 1 o piu nessun affare
# eh possibile, perche in nessun momento ask<lower oppure bid>upper.

### Modifica il metodo test_strategy aggiungendo un parametro verbose=True che quando è True fa operazioni di print mentre quando è False non stampa nulla. Ovviamente devono essere modificati anche i metodi che test_strategy utilizza qualora questi abbiano dei print. Inoltre test_strategy restituisce la performance finale.

Dopodiché crea il metodo optimize_strategy (puoi copiare gran parte del codice dalle lezioni precedenti) che ottimizza la strategia SMA continuando a chiamare test_strategy ovviamente con il parametro verbose messo pari a False, altrimenti abbiamo migliaia di righe di output. Consiglio di tenere i range stretti dato che questa volta usando un metodo iterativo è molto più lento.

In [52]:
# faccio una classe figlia della mia IterativeBase e overrido i suoi metodi (come abbiamo fatto con
# oanda nella lezione del 12.11)

In [53]:
class IterativeBaseV2(IterativeBase):

    def __init__(self, symbol, start, end, bilancio, fee=(0,0,0)):
        super().__init__(symbol,start,end,bilancio)
        #fee(x,y,z) x percentuale, comm minima, comm massima.
        self.verbose=True
        
    
    def buy(self,t,units=None,amount=None):
        if self.closure is False:
            Adate, prezzo, bid, ask, Abid, Aask=self.get_values(t+1)
            if amount is not None:
                units= int(amount/Aask)   #sto comprando
            self.current_balance-= units*Aask       #riduco i miei soldi, sarebbe unidea controllare che io cho i soldi
            commissioni= min(max(units*Aask*self.fee[0],self.fee[1]),self.fee[2])
            self.current_balance-=commissioni
            self.units+=units
            self.trades+=1
            if self.verbose is True:
                print(str(Adate),"Compro "+str(units) + ", al prezzo: " + str(Aask) + " spendendo " +str((units*Aask)))
        else:
            date, prezzo, bid, ask, Abid, Aask=self.get_values(t)
            if amount is not None:
                units= int(amount/ask)   #sto comprando
            self.current_balance-= units*ask       
            commissioni= min(max(units*ask*self.fee[0],self.fee[1]),self.fee[2])
            self.current_balance-=commissioni
            self.units+=units
            self.trades+=1
            if self.verbose is True:
                print(str(date),"Compro "+str(units) + ", al prezzo: " + str(ask) + " spendendo " +str((units*ask)))
        
    def sell(self,t,units=None,amount=None): # dato per scontato units eh positivo 
        if self.closure is False:
            Adate, prezzo, bid, ask, Abid, Aask=self.get_values(t+1)
            if amount is not None:
                units= int(amount/Abid)   
            self.current_balance+= units*Abid       
            commissioni= min(max(units*Abid*self.fee[0],self.fee[1]),self.fee[2])
            self.current_balance-=commissioni
            self.units-=units
            self.trades+=1
            if self.verbose is True:
                print(str(Adate),"Vendo "+str(units) + ", al prezzo: " + str(Abid) + " ricevendo " +str((units*Abid)))
        else:
            date, prezzo, bid, ask, Abid, Aask=self.get_values(t)
            if amount is not None:
                units= int(amount/bid)   #sto vendendo
            self.current_balance+= units*bid      
            commissioni= min(max(units*bid*self.fee[0],self.fee[1]),self.fee[2])
            self.current_balance-=commissioni
            self.units-=units
            self.trades+=1
            if self.verbose is True:
                print(str(date),"Vendo "+str(units) + ", al prezzo: " + str(bid) + " ricevendo " +str((units*bid)))
        
    def close(self,t):
        self.closure=True
        Adate, prezzo, bid, ask, Abid, Aask=self.get_values(t)
        if self.units>0:           #ho posizione lunga
            self.sell(t,units=self.units)
        else:
            self.buy(t,units=-self.units) # attenzione al meno
        performance=(self.current_balance -self.initial_balance)/self.initial_balance
        if self.verbose is True:
            print("------CHIUSO TUTO--------")
            print(str(Adate)+ " BILANCIO: " + str(self.current_balance) + " PERFORMANCE: "
                 + str(round(performance*100,2))+"%" + ", TRADE TOTALI: " + str(self.trades))
    
    def print_current_situation(self,t):
        date, prezzo, bid, ask, Abid, Aask=self.get_values(t)
        if self.units>0:
            valore=self.units*Abid
        else:
            valore=self.units*Aask
        print(str(date)+ " INVESTITI: " +str(self.units)+ " per un totale di "+ str(valore))
        

In [54]:
bc3 = IterativeBaseV2("EUR_USD","2009-01-01","2021-10-30",100000)

In [55]:
bc3.data

,bid,ask,Abid,Aask,prezzo,logRet
time,,,,,,
2008-12-31 22:00:00,1.40390,1.40490,1.39745,1.39845,1.404400,NaN
2009-01-01 22:00:00,1.39161,1.39261,1.40390,1.40490,1.392110,-0.008790
2009-01-03 22:00:00,1.39030,1.39100,1.39131,1.39231,1.390650,-0.001049
2009-01-04 22:00:00,1.36342,1.36364,1.39030,1.39100,1.363530,-0.019694
2009-01-05 22:00:00,1.35353,1.35378,1.36342,1.36364,1.353655,-0.007269
...,...,...,...,...,...,...
2021-10-24 21:00:00,1.16096,1.16115,1.16362,1.16462,1.161055,-0.003040
2021-10-25 21:00:00,1.15955,1.15973,1.16026,1.16126,1.159640,-0.001219
2021-10-26 21:00:00,1.16023,1.16043,1.15875,1.15975,1.160330,0.000595


In [56]:
class IterativeSMABacktest(IterativeBaseV2):
    
    def __init__(self,symbol,start,end,bilancio,fee=(0,0,0)):
        super().__init__(symbol,start,end,bilancio) 
        
    def go_long(self,t,units=None,amount=None): 
        if self.posizione==-1: #se eh corta debbo prima chiderla per sicurezza
            self.buy(t,units=-self.units)
        if amount is not None:
            if amount=="all":
                amount= self.current_balance
            self.buy(t,amount=amount)
        else: # oko problema
            self.buy(t,units=units)
        self.posizione=1
           

    def go_short(self,t,units=None,amount=None): 
        if self.posizione==1: #se eh corta debbo prima chiderla per sicurezza
            self.sell(t,units=self.units)
        if amount is not None:
            if amount=="all":
                amount= self.current_balance
            self.sell(t,amount=amount)
        else:
            self.sell(t,units=units)
        self.posizione=-1
    
    #do per scontato che ho gia fatto optimize e so gia quale sono i miei paraemtri ottimal        
    #Modifica il metodo test_strategy aggiungendo un parametro verbose=True 
    #che quando è True fa operazioni di print mentre quando è False non stampa nulla.
    def test_SMA_strategy(self, SMA_s, SMA_l, verbose=True):
        self.verbose=verbose
        self.posizione=0
        self.trades=0
        self.current_balance=self.initial_balance
        self.get_data()
        self.data["SMA_s"]=self.data.prezzo.rolling(SMA_s).mean()
        self.data["SMA_l"]=self.data.prezzo.rolling(SMA_l).mean()
        self.data.dropna(inplace=True)
        for t in range(len(self.data)-2): #chiudo tutto leposizione nell penultimo giorno cosi avro il presso chiusura ultimo giorno
            if self.data.SMA_s.iloc[t]>self.data.SMA_l.iloc[t]:
                if self.posizione!=1:
                    self.go_long(t,amount="all")
                    self.posizione=1
            elif self.data.SMA_s.iloc[t]<self.data.SMA_l.iloc[t]:
                if self.posizione!=-1:
                    self.go_short(t,amount="all") # oko t+1???
                    self.posizione=-1
        self.close(len(self.data)-1)        #Inoltre test_strategy restituisce la performance finale:
        if self.verbose is False:   
            return ((self.current_balance-self.initial_balance)/self.initial_balance)
        
        
#ottimizza la strategia SMA continuando a 
#chiamare test_strategy ovviamente con il parametro verbose messo pari a False,
    def optimize_parameters(self, SMA_s_range, SMA_l_range): # ranges are tuples
        combinazioni = list(product(range(SMA_s_range[0],SMA_s_range[1]),range(SMA_l_range[0],SMA_l_range[1]) ))
        risultati= []
        for comb in combinazioni:
            dummy = self.test_SMA_strategy(comb[0],comb[1],False)
            risultati.append(dummy)
        
        best_performance = np.max(risultati)
        best_combinazione = combinazioni[np.argmax(risultati)]
        
        return best_combinazione, best_performance

In [57]:
bc4 = IterativeSMABacktest("EUR_USD","2009-01-01","2021-10-30",100000)

In [59]:
bc4.test_SMA_strategy(50,200, False) 

0.23932367259999984

In [58]:
bc5 = IterativeSMABacktest("EUR_USD", "2008-10-01", "2021-10-01",1000000)

In [60]:
bc5.optimize_parameters((3,10),(100,112))

((8, 111), -0.02858378533999715)

In [61]:
bc5.optimize_parameters((1,2),(12,14))

((1, 12), -0.3385369180399956)

In [62]:
bc4.current_balance

123932.36725999998

In [ ]:
bc4.initial_balance

In [64]:
bc4.test_SMA_strategy(50,200, True) 

2009-08-07 21:00:00 Compro 70478, al prezzo: 1.41888 spendendo 99999.82463999999
2010-01-20 22:00:00 Vendo 70478, al prezzo: 1.40833 ricevendo 99256.28174
2010-01-20 22:00:00 Vendo 70478, al prezzo: 1.40833 ricevendo 99256.28174
2010-09-27 21:00:00 Compro 70478, al prezzo: 1.35862 spendendo 95752.82036
2010-09-27 21:00:00 Compro 75635, al prezzo: 1.35862 spendendo 102759.2237
2011-01-12 22:00:00 Vendo 75635, al prezzo: 1.33626 ricevendo 101068.0251
2011-01-12 22:00:00 Vendo 75635, al prezzo: 1.33626 ricevendo 101068.0251
2011-01-30 22:00:00 Compro 75635, al prezzo: 1.36946 spendendo 103579.1071
2011-01-30 22:00:00 Compro 71968, al prezzo: 1.36946 spendendo 98557.29728
2011-09-10 21:00:00 Vendo 71968, al prezzo: 1.35825 ricevendo 97750.536
2011-09-10 21:00:00 Vendo 71968, al prezzo: 1.35825 ricevendo 97750.536
2012-10-08 21:00:00 Compro 71968, al prezzo: 1.28857 spendendo 92735.80576
2012-10-08 21:00:00 Compro 79751, al prezzo: 1.28857 spendendo 102764.74607
2013-04-15 21:00:00 Vendo 79

In [65]:
bc4.current_balance

124149.04032000006

In [66]:
bc4.initial_balance

100000

In [67]:
# osservazione: bc4.test_SMA_strategy(50,200, False) e bc4.test_SMA_strategy(50,200, True) hanno 
# alla fine del processo lo steso current balance e lo stesso initial balance, ma i numeri di performance
# vengono diverso, magari un problema di arrotondamento dal computer.
# Dalle 3 strategie viste sopra, lunica profitevole e stata la SMA con 50,200 e la contrarian va anche bene per finestre 
# molto lunghe (100) e molto corte (5)

#    ignorare
def test_MC_strategy(self, isMomentum=True, window=3):
        self.posizione=0
        self.trades=0
        self.current_balance=self.initial_balance
        self.get_data()
        self.data["avglogret"]= self.data.rolling(window).logRet.mean()
        if isMomentum:
            for t in range(len(self.data)-1): 
                print("t="+str(t))
                if (self.data.iloc[t].rolling(window).mean()).logRet>0:
                    if self.posizione!=1:
                        self.go_long(t,amount="all")
                        self.posizione=1
                elif (self.data.iloc[t].rolling(window).mean()).logRet<0:
                    if self.posizione!=-1:
                        self.go_short(t,amount="all")
                        self.posizione=-1  
        else:
            for t in range(len(self.data)-1): 
                print("t="+str(t))
                if (self.data.iloc[t].rolling(window).mean()).logRet<0:
                    if self.posizione!=1:
                        self.go_long(t,amount="all")
                        self.posizione=1
                elif (self.data.iloc[t].rolling(window).mean()).logRet>0:
                    if self.posizione!=-1:
                        self.go_short(t,amount="all") # oko t+1 ???
                        self.posizione=-1  
        self.close(len(self.data)-1)